# Тестирование класса DataPreprocessor

Полностью автономная демонстрация без внешних зависимостей (только pandas + numpy).
Используется синтетический датасет с пропусками и категориальными признаками.

## 1. Импорт библиотек и создание синтетического датасета

In [1]:
import pandas as pd
import numpy as np

# Воспроизводимость результатов
np.random.seed(42)

# Создание синтетического датасета
n_samples = 200

data = {
    'age': np.random.randint(18, 70, size=n_samples),
    'income': np.random.normal(loc=50000, scale=15000, size=n_samples),
    'balance': np.random.exponential(scale=5000, size=n_samples),
    'transactions': np.random.poisson(lam=25, size=n_samples),
    'city': np.random.choice(['Moscow', 'SPB', 'Novosibirsk', 'Ekaterinburg'], size=n_samples),
    'gender': np.random.choice(['M', 'F'], size=n_samples),
    'loyalty': np.random.choice(['Bronze', 'Silver', 'Gold', 'Platinum'], size=n_samples),
    'has_loan': np.random.choice([0, 1], size=n_samples, p=[0.7, 0.3]),
}

df = pd.DataFrame(data)

# Добавление искусственных пропусков разной интенсивности
df.loc[np.random.choice(df.index, size=40), 'income'] = np.nan
df.loc[np.random.choice(df.index, size=25), 'balance'] = np.nan
df.loc[np.random.choice(df.index, size=60), 'transactions'] = np.nan  # 30% пропусков → будет удалён при threshold=0.25
df.loc[np.random.choice(df.index, size=15), 'city'] = np.nan
df.loc[np.random.choice(df.index, size=10), 'gender'] = np.nan

print("="*70)
print("СИНТЕТИЧЕСКИЙ ДАТАСЕТ (первые 12 строк)")
print("="*70)
print(df.head(12))

print("\n" + "="*70)
print("СТАТИСТИКА ПРОПУСКОВ")
print("="*70)
missing_stats = df.isnull().sum()
missing_stats = missing_stats[missing_stats > 0].sort_values(ascending=False)
for col in missing_stats.index:
    pct = missing_stats[col] / len(df) * 100
    print(f"{col:15s} | пропусков: {missing_stats[col]:3d} ({pct:5.1f}%)")

СИНТЕТИЧЕСКИЙ ДАТАСЕТ (первые 12 строк)
    age        income       balance  transactions          city gender  \
0    56  51905.183591  13795.255995          23.0        Moscow      M   
1    69           NaN    199.873886           NaN           NaN      M   
2    46  79176.734217   2705.960533           NaN        Moscow      F   
3    32  47699.953137  17144.983873          26.0           SPB      M   
4    60  36395.191619           NaN           NaN  Ekaterinburg      F   
5    25  37739.687673   2753.647685          28.0           SPB    NaN   
6    38           NaN   4202.673885          32.0           SPB      M   
7    56  41452.574774   4289.219755           NaN  Ekaterinburg      F   
8    36  18626.004955   6577.274078          25.0           SPB      M   
9    40  68956.107961    683.050517          32.0           SPB      M   
10   28           NaN   1438.520025          22.0        Moscow      M   
11   28  49589.353316   4343.983374           NaN        Moscow      M  

## 2. Базовая обработка данных через fit_transform()

In [2]:
# Импорт вашего класса
from data_preprocessor import DataPreprocessor

# Создание препроцессора и полная обработка
preprocessor = DataPreprocessor(df)
processed_df = preprocessor.fit_transform(threshold=0.25, method='minmax')

print("="*70)
print("ОБРАБОТАННЫЙ ДАТАСЕТ (первые 12 строк)")
print("="*70)
print(processed_df.head(12))

print("\n" + "="*70)
print("ОСНОВНЫЕ МЕТРИКИ РЕЗУЛЬТАТА")
print("="*70)
print(f"Исходный размер:      {df.shape[0]} строк × {df.shape[1]} столбцов")
print(f"Финальный размер:     {processed_df.shape[0]} строк × {processed_df.shape[1]} столбцов")
print(f"Пропусков до:         {df.isnull().sum().sum()}")
print(f"Пропусков после:      {processed_df.isnull().sum().sum()}")
print(f"Числовых столбцов:    {processed_df.select_dtypes(include='number').shape[1]}")

ОБРАБОТАННЫЙ ДАТАСЕТ (первые 12 строк)
         age    income   balance  transactions  has_loan  city_Ekaterinburg  \
0   0.745098  0.495586  0.499993      0.391304       0.0              False   
1   1.000000  0.492570  0.004773      0.521739       0.0              False   
2   0.549020  0.795282  0.096059      0.521739       0.0              False   
3   0.274510  0.449374  0.622008      0.521739       0.0              False   
4   0.823529  0.325142  0.135530      0.521739       0.0               True   
5   0.137255  0.339917  0.097796      0.608696       0.0              False   
6   0.392157  0.492570  0.150577      0.782609       0.0              False   
7   0.745098  0.380719  0.153730      0.521739       0.0               True   
8   0.352941  0.129870  0.237074      0.478261       0.0              False   
9   0.431373  0.682964  0.022373      0.782609       0.0              False   
10  0.196078  0.492570  0.049892      0.347826       0.0              False   
11  0.196078 

## 3. Детальный анализ преобразований

In [3]:
info = preprocessor.get_transform_info()

print("="*70)
print("ДЕТАЛЬНЫЙ АНАЛИЗ ПРЕОБРАЗОВАНИЙ")
print("="*70)

# Удалённые столбцы
print("\n🗑️  УДАЛЁННЫЕ СТОЛБЦЫ (пропусков > 25%):")
if info['removed_cols']:
    for col in info['removed_cols']:
        orig_missing = df[col].isnull().sum() / len(df) * 100
        print(f"   • {col:15s} → {orig_missing:5.1f}% пропусков")
else:
    print("   Нет удалённых столбцов")

# Заполнение пропусков
print("\n💧 ЗАПОЛНЕНИЕ ПРОПУСКОВ:")
if info['imputation']:
    for col, imp_info in info['imputation'].items():
        val = imp_info['fill_val']
        val_str = f"{val:.2f}" if isinstance(val, (int, float)) else str(val)
        orig_missing = df[col].isnull().sum()
        print(f"   • {col:15s} | {imp_info['fill_type']:6s} = {val_str:8s} | было пропусков: {orig_missing:3d}")
else:
    print("   Пропуски отсутствуют")

# One-hot encoding
print("\n🏷️  ONE-HOT КОДИРОВАНИЕ:")
print(f"   Исходные категориальные столбцы: {info['og_cols']}")
print(f"   Создано новых столбцов: {len(info['one_hot_cols'])}")
if info['one_hot_cols']:
    print("   Примеры новых столбцов:")
    for col in info['one_hot_cols'][:6]:
        print(f"      • {col}")

# Нормализация
print("\n📏 НОРМАЛИЗАЦИЯ (первые 5 столбцов):")
for i, (col, params) in enumerate(list(info['normalization'].items())[:5]):
    if params['method'] == 'minmax':
        print(f"   • {col:15s} | min={params['min']:8.2f} | max={params['max']:8.2f}")
    else:
        print(f"   • {col:15s} | mean={params['mean']:8.2f} | std={params['std']:8.2f}")

ДЕТАЛЬНЫЙ АНАЛИЗ ПРЕОБРАЗОВАНИЙ

🗑️  УДАЛЁННЫЕ СТОЛБЦЫ (пропусков > 25%):
   Нет удалённых столбцов

💧 ЗАПОЛНЕНИЕ ПРОПУСКОВ:
   • income          | median = 51630.68 | было пропусков:  39
   • balance         | median = 3789.58  | было пропусков:  24
   • transactions    | median = 26.00    | было пропусков:  48
   • city            | mode   = Moscow   | было пропусков:  15
   • gender          | mode   = F        | было пропусков:  10

🏷️  ONE-HOT КОДИРОВАНИЕ:
   Исходные категориальные столбцы: ['city', 'gender', 'loyalty']
   Создано новых столбцов: 10
   Примеры новых столбцов:
      • city_Ekaterinburg
      • city_Moscow
      • city_Novosibirsk
      • city_SPB
      • gender_F
      • gender_M

📏 НОРМАЛИЗАЦИЯ (первые 5 столбцов):
   • age             | min=   18.00 | max=   69.00
   • income          | min= 6808.15 | max=97805.50
   • balance         | min=   68.83 | max=27522.09
   • transactions    | min=   14.00 | max=   37.00
   • has_loan        | min=    0.00 | max=    1.

## 4. Тестирование различных стратегий

In [5]:
print("="*70)
print("ТЕСТ: ВЛИЯНИЕ ПОРОГА УДАЛЕНИЯ ПРОПУСКОВ")
print("="*70)
for thresh in [0.10, 0.25, 0.40]:
    prep = DataPreprocessor(df.copy())
    result = prep.fit_transform(threshold=thresh, method='minmax')
    removed = prep.get_transform_info()['removed_cols']
    print(f"threshold={thresh:.2f} | удалено: {len(removed):2d} столбцов | итог: {result.shape[1]:3d} столбцов | пропусков: {result.isnull().sum().sum()}")

print("\n" + "="*70)
print("ТЕСТ: СРАВНЕНИЕ МЕТОДОВ НОРМАЛИЗАЦИИ")
print("="*70)
for method in ['minmax', 'std']:
    prep = DataPreprocessor(df.copy())
    result = prep.fit_transform(threshold=0.25, method=method)
    col = 'age'
    if col in result.columns:
        stats = {
            'min': result[col].min(),
            'max': result[col].max(),
            'mean': result[col].mean(),
            'std': result[col].std()
        }
        print(f"{method:6s} | min={stats['min']:7.4f} | max={stats['max']:7.4f} | mean={stats['mean']:7.4f} | std={stats['std']:7.4f}")

ТЕСТ: ВЛИЯНИЕ ПОРОГА УДАЛЕНИЯ ПРОПУСКОВ
threshold=0.10 | удалено:  3 столбцов | итог:  12 столбцов | пропусков: 0
threshold=0.25 | удалено:  0 столбцов | итог:  15 столбцов | пропусков: 0
threshold=0.40 | удалено:  0 столбцов | итог:  15 столбцов | пропусков: 0

ТЕСТ: СРАВНЕНИЕ МЕТОДОВ НОРМАЛИЗАЦИИ
minmax | min= 0.0000 | max= 1.0000 | mean= 0.4985 | std= 0.2930
std    | min=-1.7016 | max= 1.7116 | mean= 0.0000 | std= 1.0000


## 6. Тестирование обработки ошибок

In [6]:
print("="*70)
print("ТЕСТИРОВАНИЕ ОБРАБОТКИ ОШИБОК")
print("="*70)

# Тест 1: Неверный тип входных данных
try:
    _ = DataPreprocessor([1, 2, 3])
    print("❌ Не обнаружена ошибка при передаче списка")
except TypeError as e:
    print(f"✅ TypeError при списке: {e}")

# Тест 2: Неверный порог удаления пропусков
try:
    prep = DataPreprocessor(df.copy())
    prep.remove_missing(threshold=1.7)
    print("❌ Не обнаружена ошибка при threshold=1.7")
except ValueError as e:
    print(f"✅ ValueError при threshold=1.7: {e}")

# Тест 3: Неверный метод нормализации
try:
    prep = DataPreprocessor(df.copy())
    prep.normalize_numeric(method='zscore')
    print("❌ Не обнаружена ошибка при методе 'zscore'")
except ValueError as e:
    print(f"✅ ValueError при методе 'zscore': {e}")

ТЕСТИРОВАНИЕ ОБРАБОТКИ ОШИБОК
✅ TypeError при списке: Wrong input data
✅ ValueError при threshold=1.7: Wrong threshold, must be between 0 and 1
✅ ValueError при методе 'zscore': Wrong method, must be minmax or std


## 7. Итоговая сводка

In [7]:
info = preprocessor.get_transform_info()

print("="*70)
print("ИТОГОВАЯ СВОДКА")
print("="*70)
print(f"\n📊 ИСХОДНЫЕ ДАННЫЕ")
print(f"   Строк:       {df.shape[0]}")
print(f"   Столбцов:    {df.shape[1]}")
print(f"   Пропусков:   {df.isnull().sum().sum()}")
print(f"   Категорий:   {len(info['og_cols'])} столбцов")

print(f"\n🧹 ОЧИСТКА")
print(f"   Удалено:     {len(info['removed_cols'])} столбцов")
print(f"   Заполнено:   {len(info['imputation'])} столбцов")

print(f"\n🏷️  КОДИРОВАНИЕ")
print(f"   Исходных:    {len(info['og_cols'])} категориальных столбцов")
print(f"   Создано:     {len(info['one_hot_cols'])} one-hot столбцов")

print(f"\n📏 НОРМАЛИЗАЦИЯ")
print(f"   Столбцов:    {len(info['normalization'])}")
print(f"   Метод:       minmax")

print(f"\n✅ РЕЗУЛЬТАТ")
print(f"   Строк:       {processed_df.shape[0]}")
print(f"   Столбцов:    {processed_df.shape[1]}")
print(f"   Пропусков:   {processed_df.isnull().sum().sum()}")
print(f"   Готов к ML:  ✅ Все признаки числовые, нормализованы")

print("\n" + "="*70)
print("✅ ПРЕПРОЦЕССОР УСПЕШНО ПРОТРЕСТИРОВАН")
print("="*70)

ИТОГОВАЯ СВОДКА

📊 ИСХОДНЫЕ ДАННЫЕ
   Строк:       200
   Столбцов:    8
   Пропусков:   136
   Категорий:   3 столбцов

🧹 ОЧИСТКА
   Удалено:     0 столбцов
   Заполнено:   5 столбцов

🏷️  КОДИРОВАНИЕ
   Исходных:    3 категориальных столбцов
   Создано:     10 one-hot столбцов

📏 НОРМАЛИЗАЦИЯ
   Столбцов:    5
   Метод:       minmax

✅ РЕЗУЛЬТАТ
   Строк:       200
   Столбцов:    15
   Пропусков:   0
   Готов к ML:  ✅ Все признаки числовые, нормализованы

✅ ПРЕПРОЦЕССОР УСПЕШНО ПРОТРЕСТИРОВАН


## Заключение

Реализованный класс `DataPreprocessor`:

✅ Работает без внешних зависимостей (только pandas + numpy)

✅ Полностью автономен — использует синтетические данные

✅ Сохраняет параметры преобразований для применения к новым данным

✅ Поддерживает настройку порогов и методов обработки

✅ Включает базовую обработку ошибок ввода

✅ Готов к использованию в production-пайплайнах подготовки данных